In [94]:
import sys

from PIL import Image

try:
    img = Image.open("images/img1.png")
except FileNotFoundError as e:
    print(e) 
    sys.exit(1)

print(img.format, img.size, img.mode)

# img.show()

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1192x420 at 0x76A0F4762690>
PNG (1192, 420) RGB


In [74]:
box = (100, 100, 1000, 300)
region = img.crop(box)
# region.show()
region.save("images/img1_crop.jpeg")

In [75]:
region_trans = region.transpose(Image.Transpose.ROTATE_180)
# region_trans.show()
region_trans.save("images/img1_trans.jpeg")

## Pasting

In [76]:
img2 = img.copy()
print(img2.width)
img.paste(region_trans, box)
# img.show()

1192


## Rolling

In [77]:
def roll(img:Image, delta):
    """Roll an image sideways"""
    xsize, ysize = img.size
    
    delta = delta % xsize
    img_result = img.copy()
    if delta == 0:
        return img_result
    
    part1 = img.crop((0, 0, delta, ysize))
    part2 = img.crop((delta, 0, xsize, ysize))
    img_result.paste(part1, (xsize-delta, 0, xsize, ysize))
    img_result.paste(part2, (0, 0, xsize-delta, ysize))
    
    return img_result

img_roll = roll(img2, 500)
# img_roll.show()

## Merging

In [78]:
def merge_x(img1:Image, img2:Image)-> Image:
    w = img1.size[0] + img2.size[0]
    h = max(img1.size[1], img2.size[1])
    img = Image.new("RGBA", (w, h))
    
    img.paste(img1)
    img.paste(img2, (img1.size[0], 0))
    
    return img

def merge_y(img1:Image, img2:Image)-> Image:
    w = max(img1.size[0], img2.size[0])
    h = img1.size[1] + img2.size[1]
    img = Image.new("RGBA", (w, h))
    
    img.paste(img1)
    img.paste(img2, (0, img1.size[1]))
    
    return img

img_merge_x = merge_x(img, img2)
# img_merge_x.show()

img_merge_y = merge_y(img, img2)
# img_merge_y.show()

## Splitting & merging bands

In [79]:
img3 = Image.open("images/img.jpg")

print(img3.size)

r, g, b = img3.split()
# r.show()
# g.show()
# b.show()
img4 = Image.merge("RGB", (b, g, r))
# img4.show()

(301, 320)


## Geometrical transforms

### Simple geometry transforms

In [80]:
img_res = img3.resize((100, 101))
# img_res.show()
img_rot = img3.rotate(45) # degrees counter-clockwise
# img_rot.show()

### Transposing

In [81]:
img_trans = img3.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
# img_trans.show()
img_trans = img3.transpose(Image.Transpose.FLIP_TOP_BOTTOM)
# img_trans.show()
img_trans = img3.transpose(Image.Transpose.ROTATE_90)
# img_trans.show()
img_trans = img3.transpose(Image.Transpose.ROTATE_180)
# img_trans.show()
img_trans = img3.transpose(Image.Transpose.ROTATE_270)
# img_trans.show()

## Color transforms

### modes conversion

In [82]:
img_l = img3.convert("L")
# img_l.show()
img_rgb = img_l.convert("RGB")
# img_rgb.show()

### Enhancement (filters)

In [83]:
from PIL import ImageFilter
img_fil = img3.filter(ImageFilter.DETAIL)
# img_fil.show("Image filtered")

### Point operations (light level)

In [84]:
img_dark = img3.point(lambda i: i * 0.2)
img_light = img3.point(lambda k: k * 6)
# img_dark.show()
# img_light.show()

### Processing individual bands

In [85]:
# split the image into individual bands
source = img3.split()

R, G, B = 0, 1, 2

# select regions where red is less than 100
mask = source[R].point(lambda i: i < 100 and 255)
# source[R].show()
# mask.show()

# process the green band
img_g_dark = source[G].point(lambda i: i * 0.7)

# paste the processed band backn=, but only where red was < 100
source[G].paste(img_g_dark, None, mask)

# build a new multiband image
img_build = Image.merge(img3.mode, source)
# img_build.show()

## Enhancing images

In [86]:
from PIL import ImageEnhance

enh = ImageEnhance.Contrast(img3)
# enh.enhance(1.3).show("30 % more contrast")

## Image sequences

In [87]:
with Image.open("images/gif.gif") as gif:
    gif.seek(1) # skip to the 2nd frame
    try:
        img_nb = 1
        while True:
            gif.save(f"images/gifs/gif{img_nb}.png", "png")
            img_nb += 1
            gif.seek(gif.tell() + 1)
    except EOFError:
        pass # end of sequence

### with ImageSequence Iterator

In [92]:
from PIL import ImageSequence

with Image.open("images/gif.gif") as gif:
    img_nb = 0
    
    for frame in ImageSequence.Iterator(gif):
        frame.save(f"images/gifs2/gif{img_nb}.png", "png")
        img_nb += 1

## Reading an image from URL

In [72]:
from urllib.request import urlopen

url = "https://python-pillow.org/assets/images/pillow-logo.png"
img4 = Image.open(urlopen(url))
img4.show()

KeyboardInterrupt: 

## Batch Processing

### with glob

In [89]:
import glob, os

def compress_image(src_path, dest_path):
    with Image.open(src_path) as img:
        if img.mode != "RGB":
            img = img.convert("RGB")
        img.save(dest_path, "jpeg", optimize=True, quality=70)
        
paths = glob.glob("images/batch/*.*")
# print(paths)
for path in paths:
    parent_dir, file = os.path.split(path)
    compress_image(path, parent_dir + "/compressed/" + os.path.splitext(file)[0] + ".jpg")

### with pathlib

In [64]:
from pathlib import Path

paths = Path("images/batch").glob("*.*")
for path in paths:
    parent_dir, file = os.path.split(path)
    compress_image(path, parent_dir + "/compressed/" + os.path.splitext(file)[0] + ".jpg")

## Draft (only for JPEG and MPO files)

In [65]:
with Image.open("images/img.jpg") as img:
    print("original =", img.mode, img.size)
    
    img.draft("L", (100, 100))
    print("draft =", img.mode, img.size)

original = RGB (301, 320)
draft = L (151, 160)
